In [ ]:
!pip install transformers
!pip install datasets
!pip install --upgrade --no-cache-dir gdown==4.5.4

!gdown 18oZZ4jqRK-uF-Nz6ftRdgNjKix88hrnO
!unzip data_and_models.zip && rm data_and_models.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 70.1 MB/s 
     |████████████████████████████████| 163 kB 83.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.7 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 115 kB 78.2 MB/s 
     |████████████████████████████████| 127 kB 80.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google

In [ ]:
from collections import defaultdict
import csv
import random
import time

from datasets import load_metric
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import RobertaTokenizerFast


def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  
directory = "./data_and_models/"
mlength = 512
start = time.time()
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

class PSCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def top_k_accuracy(top_k, predictions, labels):
  assert len(predictions) == len(labels)
  total = 0
  correct = 0
  for i in range(len(predictions)):
    total += 1
    prediction = []
    for j, k in enumerate(predictions[i]):
      prediction.append([j, k])
    prediction.sort(key = lambda x: -x[1])
    for j, _ in prediction[:top_k]:
      if j == labels[i]:
        correct += 1
        break
  return correct/total

tasks = {
    "44": {
        "number_of_labels": 42,
         "label_column": 1,
    },
    "8": {
        "number_of_labels": 8,
        "label_column": 2,
    }
}

def compute_task(task):
  results = {}
  for train_size in [200, 300, 400]:
    t1 =[]
    seeds = range(11, 16)
    nclasses = tasks[task]["number_of_labels"]
    tasks[task]["label_column"]
    epochs = 20
    learning_rate = 4e-5
    for seed in seeds:
      np.random.seed(seed)
      torch.manual_seed(seed)
      random.seed(seed)

      index = -1
      classes = {}
      texts = []
      labels = []
      lm_reverse_mapper = {}
      with open(directory + "target_corpus.csv") as doc:
        reader = csv.reader(doc)
        next(reader)
        for row in reader:
          topic = row[tasks[task]["label_column"]]
          if topic not in classes:
            index += 1
            classes[topic] = index
            lm_reverse_mapper[index] = topic.capitalize()
          labels.append(classes[topic])
          texts.append(row[0])
      print("# classes", len(classes))
      test_size = (4165 - 400)//2 + 1
      dev_size = (4165 - 400)//2
      X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=test_size, random_state=seed)
      X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=dev_size, random_state=seed)
      if train_size < 400: 
        _, X_train, _, y_train = train_test_split(X_train, y_train, test_size = train_size, random_state = seed)
      print(len(X_train), len(X_dev), len(X_test))
      print("# classes in train", len(set(y_train)))
      print("# classes in dev", len(set(y_dev)))
      print("# classes in test", len(set(y_test)))


      train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=mlength)
      dev_encodings = tokenizer(X_dev, truncation=True, padding=True, max_length = mlength)
      test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length= mlength)


      train_dataset = PSCDataset(train_encodings, y_train)
      dev_dataset = PSCDataset(dev_encodings, y_dev)
      test_dataset = PSCDataset(test_encodings, y_test)

      from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer

      training_args = TrainingArguments(
          output_dir="./results",          # output directory
          num_train_epochs=epochs,         # total number of training epochs
          per_device_train_batch_size=16,  # batch size per device during training
          per_device_eval_batch_size=64,   # batch size for evaluation
          warmup_steps=0,                  # number of warmup steps for learning rate scheduler
          weight_decay=0.01,               # strength of weight decay
          logging_dir='./logs',            # directory for storing logs
          logging_steps=10,
          learning_rate = learning_rate,
          save_strategy= "epoch",
          evaluation_strategy="epoch",
          load_best_model_at_end= True,
          metric_for_best_model="accuracy",
          save_total_limit = 2,
          seed = seed, 
      )

      def model_init():
          return RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=nclasses)
      trainer = Trainer(
          model_init=model_init,               # the instantiated 🤗 Transformers model to be trained
          args=training_args,                  # training arguments, defined above
          train_dataset=train_dataset,         # training dataset
          eval_dataset=dev_dataset,            # evaluation dataset
          compute_metrics=compute_metrics,     # compute_metrics
          )

      trainer.train()
      predictions = trainer.predict(test_dataset)
      preds = np.argmax(predictions.predictions, axis=-1)

      t1.append(top_k_accuracy(1, predictions.predictions, test_dataset.labels))

    results[train_size] = t1
  return results

outputs = {}
for task in tasks:
  outputs[task] = compute_task(task)

np.save( directory + "figure_1_results.npy", outputs)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

# classes 42
200 1882 1883
# classes in train 34
# classes in dev 40
# classes in test 39


...

***** Running Evaluation *****
  Num examples = 1882
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-25
Configuration saved in ./results/checkpoint-25/config.json
Model weights saved in ./results/checkpoint-25/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-475] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1882
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-50
Configuration saved in ./results/checkpoint-50/config.json
Model weights saved in ./results/checkpoint-50/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1882
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-75
Configuration saved in ./results/checkpoint-75/config.json
Model weights saved in ./results/checkpoint-75/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-25] due to args.save_total_limit
***** Running Evaluatio

# classes 8
400 1882 1883
# classes in train 8
# classes in dev 8
# classes in test 8


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,

Epoch,Training Loss,Validation Loss,Accuracy
1,1.987500,1.928288,0.200850
2,1.612800,1.566792,0.448990
3,1.324800,1.462904,0.498937
4,0.941100,1.459053,0.539851
5,0.604500,1.508209,0.538789
6,0.442300,1.749491,0.520191
7,0.345300,2.000509,0.502657
8,0.157900,2.102963,0.525505
9,0.037100,2.146791,0.554729
10,0.128200,2.391114,0.535069


***** Running Evaluation *****
  Num examples = 1882
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-25
Configuration saved in ./results/checkpoint-25/config.json
Model weights saved in ./results/checkpoint-25/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1882
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-50
Configuration saved in ./results/checkpoint-50/config.json
Model weights saved in ./results/checkpoint-50/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1882
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-75
Configuration saved in ./results/checkpoint-75/config.json
Model weights saved in ./results/checkpoint-75/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-25] due to args.save_total_limit
***** Running Evaluatio

In [ ]:
for task in tasks:
  print(task + "-Topic Classification")
  results = outputs[task]
  for train_size in results:
    print(train_size, np.mean(results[train_size]), np.std(results[train_size]))

44-Topic Classification
200 0.3938396176314392 0.014272184414012389
300 0.4302708443972385 0.018305733848378577
400 0.44503451938396166 0.011139764717686148
8-Topic Classification
200 0.5348911311736589 0.018559724542714137
300 0.5472118959107807 0.015179984895792472
400 0.5595326606479023 0.016832743779634632


In [ ]:
end = time.time()
print(f"The program took {(end - start) // 60} minutes in total.")

The program took 245.0 minutes in total.


In [ ]:
from google.colab import runtime
runtime.unassign()